### To reduce unnecessary logging

In [8]:
!export TOKENIZERS_PARALLELISM=false 
import logging
logging.getLogger('httpx').setLevel(logging.WARNING)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### Import all relevant modules

In [2]:
from langserve import RemoteRunnable
import json

from commons.constants import API_BASE_URL
from llm_chain.query_helper.openai_client import OpenAIClient, model
from llm_chain.llm_config import LANGSERVE_PORT
from helper_funcs.openai_helper import OpenAiToolsHelper
from helper_funcs.embedding_helper import EmbeddingHelper
from helper_funcs.api_helper import ApiHelper

### Create prompt templates to use in the main loop

In [17]:
assistant_response_template = """Answer : {intro_text}

Response from endpoint : {endpoint}
{api_response}

_______________________
The above data follows this response format : 
{resp_format}

____________________________

"""

final_user_message = """
Given all the details discussed previosuly in this conversation, answer this questions:

Question : {complex_user_query}"""


### Get map of tool name to endpoint and response_format 

In [3]:
with open('endpoint_and_resp_format.json','r') as f:
    endpoint_and_resp_format_data = json.load(f)

### Load the Complex User Query to ask 

In [4]:
user_query = "List the urgent tickets assigned all active employees in California"
query_decomposer_input = {'question':user_query}

### Decompose it to multiple sub queries 

In [5]:
api = RemoteRunnable("http://127.0.0.1:1111/query_breaker")
sub_queries = api.invoke(query_decomposer_input)

INFO:httpx:HTTP Request: POST http://127.0.0.1:1111/query_breaker/invoke "HTTP/1.1 200 OK"


In [7]:
sub_queries

[{'sub_query': 'Get me a list of all active employees in California.',
  'category': 'hris',
  'answer_intro': 'The list of all active employees in California is:'},
 {'sub_query': 'What are the urgent tickets assigned to the active employees in California?',
  'category': 'ticketing',
  'answer_intro': 'The urgent tickets assigned to active employees in California are:'}]

In [6]:
open_ai = OpenAIClient()

In [24]:
oai_messages = []
for sub_query in sub_queries:
    sub_q = sub_query["sub_query"]
    category = sub_query["category"]
    intro = sub_query["answer_intro"]

    print("Current Subquery is : ", sub_q, " , Category : ", category)
    tools = EmbeddingHelper.get_relevant_merge_apis(sub_q, category, num_results=10)
    oai_messages.append({'role':'user','content':sub_q})

    endpoint_with_params = open_ai.get_endpoints_and_params_from_openai(
        oai_messages, tools
    )

    func_name = endpoint_with_params.choices[0].message.tool_calls[0].function.name
    arguments = json.loads(
        endpoint_with_params.choices[0].message.tool_calls[0].function.arguments
    )

    endpoint = endpoint_and_resp_format_data[func_name]["endpoint"]
    response_format = endpoint_and_resp_format_data[func_name]["response_format"]

    curr_response = ApiHelper.get_response_from_merge(endpoint, arguments)

    curr_assistant_message = assistant_response_template.format(
        subquestion=sub_q,
        intro_text=intro,
        endpoint=endpoint,
        api_response=curr_response,
        resp_format=response_format,
    )
    
    oai_messages.append({'role':'assistant','content':curr_assistant_message})


Current Subquery is :  List all active employees in California.  , Category :  hris
INFO:helper_funcs.api_helper:
----------- MERGE ENPOINT AND PARAM DETAILS -------------

INFO:helper_funcs.api_helper:Calling Merge endpoint : https://api.merge.dev/api/hris/v1/employees 
INFO:helper_funcs.api_helper:With Params : {'employment_status': 'ACTIVE', 'work_location_id': 'California'} 

INFO:helper_funcs.api_helper:
---------------------------------------------------------

Current Subquery is :  List all urgent tickets assigned to active employees in California.  , Category :  ticketing
INFO:helper_funcs.api_helper:
----------- MERGE ENPOINT AND PARAM DETAILS -------------

INFO:helper_funcs.api_helper:Calling Merge endpoint : https://api.merge.dev/api/ticketing/v1/tickets 
INFO:helper_funcs.api_helper:With Params : {'priority': 'URGENT'} 

INFO:helper_funcs.api_helper:
---------------------------------------------------------



### Save this example conversation

In [26]:
oai_messages.append({'role':'user', 'content' : final_user_message.format(complex_user_query=user_query)})
with open('example_notebook_final_prompt.json','w') as f:
    json.dump(oai_messages,f)

### Using the complete conversation, make the final OpenAI call with all context and the original complete question

In [27]:
oai_final_res = open_ai.get_response(oai_messages)
print("The answer is  : ", oai_final_res.choices[0].message.content)

The answer is  :  To list the urgent tickets assigned to all active employees in California, you would typically follow these steps:

1. **Fetch Active Employees in California**: Retrieve the list of all active employees in California.
2. **Fetch Urgent Tickets**: Retrieve the list of all tickets with a priority of "URGENT".
3. **Filter Tickets by Assignees**: Filter the urgent tickets to include only those assigned to the active employees in California.

However, based on the responses provided earlier, it seems that the API keys used to fetch the data are invalid. Therefore, I cannot provide the actual data. Below is a conceptual approach to how you would achieve this if the API keys were valid:

### Step 1: Fetch Active Employees in California
```http
GET /hris/v1/employees?employment_status=ACTIVE&location=California
```
### Step 2: Fetch Urgent Tickets
```http
GET /ticketing/v1/tickets?priority=URGENT
```
### Step 3: Filter Tickets by Assignees
After fetching the data from the abo